# Intro to TensorFlow

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np

**My first computation Graph**

In [ ]:
# Defining a graph (a default graph)
# Every declared node is automatically added 
# to the 'default-graph'
x = tf.Variable(3, name="x")
y = tf.Variable(5, name="y")
f = x * x * y + y + 2
# Run the graph inside a session.
# With the 'with' command, the session is set as the
# default sesion
with tf.Session() as sess:
    # We need to initialize the variables before
    # performing operations using them
    x.initializer.run() # Equivalent to: tf.get_default_session().run(x.initializer)
    y.initializer.run() # Equivalent to: tf.get_default_session().run(y.initializer)
    result = f.eval()   # Equivalent to: tf.get_default_session().run(f)
    
print(result)

In [ ]:
# Removing every node inside the
# default graph
tf.reset_default_graph()

# **CONSTRUCTION PHASE**
x = tf.Variable(3, name="x")
y = tf.Variable(5, name="y")
f = x * x * y + y + 2

# Add to the graph a step to initialize all variables
# (we are not actually initializing the variables in this step)
init = tf.global_variables_initializer()

# **EXECUTION PHASE**
with tf.Session() as ses:
    init.run()
    result = f.eval()

print(result)

### Evaluating nodes

In [ ]:
tf.reset_default_graph()
# Whenever we evaluate a node, Tensorflow automatically
# determines the set of nodes that it depends on.

w  = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

init = tf.global_variables_initializer()
# **The ineficient way to evaluate a set of nodes**
# By evaluating (y, z) (nodes) the following way, TensorFlow
# has to compute 'w' and 'x' twice in order to obtain (x, z)
with tf.Session() as sess:
    init.run()
    print(y.eval())
    print(z.eval())
    
tf.reset_default_graph()
w  = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

print()
init = tf.global_variables_initializer()
# **The proper way to evaluate a set of nodes**
with tf.Session() as sess:
    init.run()
    xres, yres = sess.run([y, z])
    print(xres)
    print(yres)

Operations in TensorFlow are called *ops*, `tf.constant` and `tf.Variable` are called *source ops* since they take no input. *Ops* with $n$ inputs and $m$ outputs are called *tensors*

#### Computing $\theta^\star$ with the normal equation

In [ ]:
tf.reset_default_graph()
housing = pd.read_csv("../input/housing.csv").dropna()
m, n = housing.shape
housing_data_bias = np.c_[np.ones((m, 1)), housing.drop(["ocean_proximity", "median_house_value"], axis=1).values]

# Defining the computation graph 
X = tf.constant(housing_data_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.median_house_value.values.reshape(-1, 1), dtype=tf.float32, name="y")
Xtranspose = tf.transpose(X)
theta = tf.matrix_inverse(Xtranspose @ X) @ Xtranspose @ y

with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    #print(sess.list_devices())
    theta_star = theta.eval()
    sess.run(theta)

#### Computing $\theta^\star$ with Gradient Descent